In [1]:
import os

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder \
    .appName("SragGold") \
    .master("local[*]") \
    .getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [3]:
spark.sparkContext.setLogLevel("ERROR")

In [9]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../")) 
data_path = os.path.join(base_dir, "data/silver/srag.parquet")
df = spark.read.parquet(
    data_path
)

In [12]:
sinais_sintomas = df.groupby(
    'DT_NOTIFIC'
).agg(
    F.sum(F.when(F.col('FEBRE') == 1, 1).otherwise(0)).alias('FEBRE'),
    F.sum(F.when(F.col('TOSSE') == 1, 1).otherwise(0)).alias('TOSSE'),
    F.sum(F.when(F.col('GARGANTA') == 1, 1).otherwise(0)).alias('DOR_GARGANTA'),
    F.sum(F.when(F.col('DISPNEIA') == 1, 1).otherwise(0)).alias('DISPNEIA'),
    F.sum(F.when(F.col('DESC_RESP') == 1, 1).otherwise(0)).alias('DESC_RESP'),
    F.sum(F.when(F.col('SATURACAO') == 1, 1).otherwise(0)).alias('SATURACAO'),
    F.sum(F.when(F.col('DIARREIA') == 1, 1).otherwise(0)).alias('DIARREIA'),
    F.sum(F.when(F.col('VOMITO') == 1, 1).otherwise(0)).alias('VOMITO'),
    F.sum(F.when(F.col('DOR_ABD') == 1, 1).otherwise(0)).alias('DOR_ABD'),
    F.sum(F.when(F.col('FADIGA') == 1, 1).otherwise(0)).alias('FADIGA'),
    F.sum(F.when(F.col('PERD_OLFT') == 1, 1).otherwise(0)).alias('PERD_OLFT'),
    F.sum(F.when(F.col('PERD_PALA') == 1, 1).otherwise(0)).alias('PERD_PALA'),
    F.sum(F.when(F.col('OUTRO_SIN') == 1, 1).otherwise(0)).alias('OUTRO_SIN'),
)
